In [2]:
import sys; sys.path.insert(0,'..')
from bechdeltest import *
from convokit import Corpus

In [3]:
def strip_punct(token):
    from string import punctuation
    return token.strip(punctuation)

In [4]:
fn='/Users/ryan/.bechdel/data/scripts_csv/The-Matrix.csv'
fnid=os.path.splitext(os.path.basename(fn))[0]
os.path.exists(fn)

True

In [22]:
df=pd.read_csv(fn).fillna('')

def clean_speaker_name(speaker):
    words=[w for w in strip_punct(speaker.strip()).split() if w==w.upper()]
    return ' '.join(words)


df['speaker'] = df['speaker'].apply(clean_speaker_name)
df['speaker_name'] = df['speaker'].apply(lambda x: x.title())
df['speaker_id'] = df['speaker_name'].apply(lambda x: f'{fnid}.{x.replace(" ","-")}')
df['scene_id'] = df['scene_num'].apply(lambda scene: f'{fnid}.scene_{scene:04}')
df['speech_id'] = df['line_num'].apply(lambda scene: f'{fnid}.speech_{scene:04}')
df

,line_num,speaker,speech,direction,narration,scene_desc,scene_num,num_words,speaker_name,speaker_id,scene_id,speech_id
0,1,BIG COP,Freeze!,,The hotel was abandoned after a fire licked it...,INT. HEART O' THE CITY HOTEL - NIGHT,1,1,Big Cop,The-Matrix.Big-Cop,The-Matrix.scene_0001,The-Matrix.speech_0001
1,2,BIG COP,Hands behind your head! it!,,The room is almost devoid of furniture. There ...,INT. HEART O' THE CITY HOTEL - NIGHT,1,5,Big Cop,The-Matrix.Big-Cop,The-Matrix.scene_0001,The-Matrix.speech_0002
2,3,AGENT SMITH,LIEUTENANT,,A black sedan with tinted windows glides in th...,EXT. HEART O' THE CITY HOTEL - NIGHT,2,1,Agent Smith,The-Matrix.Agent-Smith,The-Matrix.scene_0002,The-Matrix.speech_0003
3,4,AGENT SMITH,"Lieutenant, you were given",,THE MATRIX - Rev. 3/9/98,EXT. HEART O' THE CITY HOTEL - NIGHT,2,4,Agent Smith,The-Matrix.Agent-Smith,The-Matrix.scene_0002,The-Matrix.speech_0004
4,5,-LIEUTENANT,I'm just doing my job. You gimme that Juris-my...,,,EXT. HEART O' THE CITY HOTEL - NIGHT,2,18,-Lieutenant,The-Matrix.-Lieutenant,The-Matrix.scene_0002,The-Matrix.speech_0005
...,...,...,...,...,...,...,...,...,...,...,...,...
721,722,NEO,I imagine you can also feel me. The numbers be...,V.O.,,INT. HOVERCRAFT,223,14,Neo,The-Matrix.Neo,The-Matrix.scene_0223,The-Matrix.speech_0722
722,723,NEO,You won't have to search for me anymore. I'm d...,V.O.,,INT. HOVERCRAFT,223,29,Neo,The-Matrix.Neo,The-Matrix.scene_0223,The-Matrix.speech_0723
723,724,NEO,"I believe deep down, we both want this world t...",V.O.,,INT. HOVERCRAFT,223,64,Neo,The-Matrix.Neo,The-Matrix.scene_0223,The-Matrix.speech_0724
724,725,NEO,"When I used to look out at this world, all I c...",V.O.,,INT. HOVERCRAFT,223,64,Neo,The-Matrix.Neo,The-Matrix.scene_0223,The-Matrix.speech_0725


In [23]:
## Create conversations df

ld_scenes=[]
for scene_id, df_scene in sorted(df.groupby('scene_id')):
    scene_desc = df_scene.scene_desc.iloc[0]
    scene_num = df_scene.scene_num.iloc[0]
    scene_int = scene_desc.upper().startswith('INT. ')
    scene_ext = scene_desc.upper().startswith('EXT. ')
    scene_len = len(df_scene)
    scene_num_words = sum(df_scene.num_words)
    scene_meta={
        'id':scene_id,
        'id_fn':fnid,
        'meta.scene_desc':scene_desc,
        'meta.scene_int':scene_int,
        'meta.scene_ext':scene_int,
        'meta.convo_num_speeches':scene_len,
        'meta.convo_num_words':scene_num_words,
        'meta.scene_includes':'; '.join(df_scene.speaker_name.unique())
    }
    ld_scenes.append(scene_meta)
df_convos = pd.DataFrame(ld_scenes)#.set_index('id')
df_convos

,id,id_fn,meta.scene_desc,meta.scene_int,meta.scene_ext,meta.convo_num_speeches,meta.convo_num_words,meta.scene_includes
0,The-Matrix.scene_0001,The-Matrix,INT. HEART O' THE CITY HOTEL - NIGHT,True,True,2,6,Big Cop
1,The-Matrix.scene_0002,The-Matrix,EXT. HEART O' THE CITY HOTEL - NIGHT,False,False,7,65,Agent Smith; -Lieutenant; Lieutenant
2,The-Matrix.scene_0005,The-Matrix,INT. HEART O' THE CITY HOTEL,True,True,5,57,Man Trinity; Morpheus; Trinity; Morpheus Trini...
3,The-Matrix.scene_0008,The-Matrix,EXT. ROOF,False,False,2,114,Cop
4,The-Matrix.scene_0010,The-Matrix,INT. BACK STAIRWELL,True,True,1,21,Trinity
...,...,...,...,...,...,...,...,...
142,The-Matrix.scene_0213,The-Matrix,INT. MAIN DECK,True,True,1,53,Trinity
143,The-Matrix.scene_0216,The-Matrix,INT. HALL - DAY,True,True,1,117,Neo
144,The-Matrix.scene_0217,The-Matrix,INT. MAIN DECK,True,True,1,4,Tank Morpheus
145,The-Matrix.scene_0221,The-Matrix,INT. MAIN DECK,True,True,1,4,Trinity


In [24]:
## Create SPEAKER dataframe

ld_speakers=[]
for speaker_id,df_speaker in df.groupby('speaker_id'):
    # if len(df_speaker)<2: continue
    odx = {
        'id':speaker_id,
        'meta.speaker':df_speaker.iloc[0].speaker,
        'meta.speaker_name':df_speaker.iloc[0].speaker_name,
        'meta.speaker_num_speeches':len(df_speaker),
        'meta.speaker_num_words':df_speaker.num_words.sum(),
        # 'meta.speaker_avg_num_words_per_speech':len(df_speaker) / df_speaker.num_words.sum(),
    }
    ld_speakers.append(odx)
df_speakers = pd.DataFrame(ld_speakers)#.set_index('id').sort_values('meta.speaker_num_words',ascending=False)
df_speakers

,id,meta.speaker,meta.speaker_name,meta.speaker_num_speeches,meta.speaker_num_words
0,The-Matrix.-Apoc,-APOC,-Apoc,1,8
1,The-Matrix.-Cop,-COP,-Cop,1,4
2,The-Matrix.-Cypher,-CYPHER,-Cypher,1,3
3,The-Matrix.-Lieutenant,-LIEUTENANT,-Lieutenant,1,18
4,The-Matrix.915!A.M,915!A.M,915!A.M,1,1
...,...,...,...,...,...
91,The-Matrix.Trinity-I-Morpheus,TRINITY I MORPHEUS,Trinity I Morpheus,1,28
92,The-Matrix.Trinity-Morpheus,TRINITY MORPHEUS,Trinity Morpheus,1,2
93,The-Matrix.Trinity-Neo,TRINITY NEO,Trinity Neo,5,40
94,The-Matrix.Trinity-Switch,TRINITY SWITCH,Trinity Switch,1,7


In [25]:
## Create UTTERANCE dataframe
ld_utts = []
for i,row in df.iterrows():
    utt_d = {
        'id':row.speech_id,
        'timestamp':row.line_num,
        'text':row.speech,
        'speaker':row.speaker_id,
        'reply_to':None,
        'conversation_id':row.scene_id,
        'meta.speech_direction':row.direction,
        'meta.speech_narration':row.narration,
        'meta.speech_num_words':row.num_words
    }
    ld_utts.append(utt_d)
df_utts=pd.DataFrame(ld_utts)#.set_index('id')
df_utts

,id,timestamp,text,speaker,reply_to,conversation_id,meta.speech_direction,meta.speech_narration,meta.speech_num_words
0,The-Matrix.speech_0001,1,Freeze!,The-Matrix.Big-Cop,None,The-Matrix.scene_0001,,The hotel was abandoned after a fire licked it...,1
1,The-Matrix.speech_0002,2,Hands behind your head! it!,The-Matrix.Big-Cop,None,The-Matrix.scene_0001,,The room is almost devoid of furniture. There ...,5
2,The-Matrix.speech_0003,3,LIEUTENANT,The-Matrix.Agent-Smith,None,The-Matrix.scene_0002,,A black sedan with tinted windows glides in th...,1
3,The-Matrix.speech_0004,4,"Lieutenant, you were given",The-Matrix.Agent-Smith,None,The-Matrix.scene_0002,,THE MATRIX - Rev. 3/9/98,4
4,The-Matrix.speech_0005,5,I'm just doing my job. You gimme that Juris-my...,The-Matrix.-Lieutenant,None,The-Matrix.scene_0002,,,18
...,...,...,...,...,...,...,...,...,...
721,The-Matrix.speech_0722,722,I imagine you can also feel me. The numbers be...,The-Matrix.Neo,None,The-Matrix.scene_0223,V.O.,,14
722,The-Matrix.speech_0723,723,You won't have to search for me anymore. I'm d...,The-Matrix.Neo,None,The-Matrix.scene_0223,V.O.,,29
723,The-Matrix.speech_0724,724,"I believe deep down, we both want this world t...",The-Matrix.Neo,None,The-Matrix.scene_0223,V.O.,,64
724,The-Matrix.speech_0725,725,"When I used to look out at this world, all I c...",The-Matrix.Neo,None,The-Matrix.scene_0223,V.O.,,64


In [26]:
from convokit import Corpus
new_corpus = Corpus.from_pandas(
    utterances_df=df_utts,
    speakers_df=df_speakers,
    conversations_df=df_convos
)
new_corpus

726it [00:00, 26235.86it/s]


In [30]:
new_corpus.print_summary_stats()

Number of Speakers: 96
Number of Utterances: 726
Number of Conversations: 147


In [28]:
convo=new_corpus.get_conversation('The-Matrix.scene_0021')

In [29]:
convo.get_chronological_utterance_list()

[Utterance({'obj_type': 'utterance', 'meta': {'speech_direction': '', 'speech_narration': 'Screaming, Neo bolts upright in bed. He realizes that he is home. Was it a dream? His mouth is normal. His stomach looks fine. He starts to take a deep, everything-is-okay breath when -The PHONE RINGS. It almost stops his heart. It continues RINGING, building pressure in the room, forcing him up out of bed, sucking him in with an almost gravitational force. He answers it, saying nothing. THE MATRIX - Rev. 3/9/98', 'speech_num_words': 12}, 'vectors': [], 'speaker': Speaker({'obj_type': 'speaker', 'meta': {}, 'vectors': [], 'owner': None, 'id': 'The-Matrix.Morpheus'}), 'conversation_id': 'The-Matrix.scene_0021', 'reply_to': None, 'timestamp': 116, 'text': 'This line is tapped so I must be brief. The Agents --', 'owner': <convokit.model.corpus.Corpus object at 0x2a9b21ac0>, 'id': 'The-Matrix.speech_0116'}),
 Utterance({'obj_type': 'utterance', 'meta': {'speech_direction': 'V.O.', 'speech_narration':